# Mobile App for Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The data set has data for 3,665 drawings, dating from 1982 to 2018.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

#### Auxiliary functions

We will start by defining functions that will help us throughout the probabilities calculation: `factorial` and `combinations`.

In [2]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

In [7]:
factorial(5)

120

In [14]:
def combinations(n,k):
    return (factorial(n)) / (factorial(k) * factorial(n-k))

In [16]:
combinations(5,2)

10.0

## First app version:

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [54]:
def one_ticket_probability(numbers):
    total_outcomes = combinations(49,6)
    successful_outcomes = 1
    probability = successful_outcomes / total_outcomes
    print('Option 1: Betting on these numbers you would win the big prize {} out of 100,000,000 (100 million) times!\nOption 2: Betting on these numbers you would win the big prize 1 out of {:,} times!'.format(int(round(probability*100000000,0)),int(1/probability)))

In [55]:
one_ticket_probability([1,2,3,4,5,6])

Option 1: Betting on these numbers you would win the big prize 7 out of 100,000,000 (100 million) times!
Option 2: Betting on these numbers you would win the big prize 1 out of 13,983,816 times!


#### Choosing the format

Raw probability would have been printed as:

In [39]:
total_outcomes = combinations(49,6)
successful_outcomes = 1
probability = successful_outcomes / total_outcomes
print(probability)

7.151123842018516e-08


which is not a very user friendly way to show what we mean. Therefore, we opted to write the function as the number of times someone would win the big prize. We will keep option 2.

In [105]:
def one_ticket_probability(numbers):
    total_outcomes = combinations(49,6)
    successful_outcomes = 1
    probability = successful_outcomes / total_outcomes
    print('Betting on these numbers you would win the big prize 1 out of {:,} times!'.format(int(1/probability)))

## Checking historical data

We'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset).

The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

- NUMBER DRAWN 1
- NUMBER DRAWN 2
- NUMBER DRAWN 3
- NUMBER DRAWN 4
- NUMBER DRAWN 5
- NUMBER DRAWN 6

We will start by exploring the dataframe.

In [56]:
hist = pd.read_csv('649.csv')
hist.shape

(3665, 11)

In [59]:
hist.loc[[0,3664]]

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


#### Creating an extraction function

This function will receive a row from the dataset and will return a Python set with the six winning numbers for that extraction.

In [65]:
hist.iloc[0,4]

3

In [89]:
numbers_columns = range(4,10)
def extract_numbers(row):
    numbers_drawn = []
    for i in numbers_columns:
        numbers_drawn.append(row[i])
    return set(numbers_drawn)

We will create a new column on the DataFrame with all the winning numbers as a Python set.

In [90]:
hist['ALL NUMBERS'] = hist.apply(extract_numbers,axis=1)

In [91]:
hist

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER,ALL NUMBERS
0,649,1,0,6/12/1982,3,11,12,14,41,43,13,"{3, 41, 11, 12, 43, 14}"
1,649,2,0,6/19/1982,8,33,36,37,39,41,9,"{33, 36, 37, 39, 8, 41}"
2,649,3,0,6/26/1982,1,6,23,24,27,39,34,"{1, 6, 39, 23, 24, 27}"
3,649,4,0,7/3/1982,3,9,10,13,20,43,34,"{3, 9, 10, 43, 13, 20}"
4,649,5,0,7/10/1982,5,14,21,31,34,47,45,"{34, 5, 14, 47, 21, 31}"
5,649,6,0,7/17/1982,8,20,21,25,31,41,33,"{8, 41, 20, 21, 25, 31}"
6,649,7,0,7/24/1982,18,25,28,33,36,42,7,"{33, 36, 42, 18, 25, 28}"
7,649,8,0,7/31/1982,7,16,17,31,40,48,26,"{7, 40, 16, 17, 48, 31}"
8,649,9,0,8/7/1982,5,10,23,27,37,38,33,"{37, 5, 38, 10, 23, 27}"
9,649,10,0,8/14/1982,4,15,30,37,46,48,3,"{4, 37, 46, 15, 48, 30}"


#### Checking historical occurrence

We will create a function to check whether a specific combination has ever occured in history.

In [93]:
hist['ALL NUMBERS'] == {3, 41, 11, 12, 43, 14}

0        True
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3635    False
3636    False
3637    False
3638    False
3639    False
3640    False
3641    False
3642    False
3643    False
3644    False
3645    False
3646    False
3647    False
3648    False
3649    False
3650    False
3651    False
3652    False
3653    False
3654    False
3655    False
3656    False
3657    False
3658    False
3659    False
3660    False
3661    False
3662    False
3663    False
3664    False
Name: ALL NUMBERS, Length: 3665, dtype: bool

In [108]:
def check_historical_occurence(numbers,history):
    numbers_set = set(numbers)
    num_of_occurences = (history == numbers_set).sum()
    total_extractions = len(history)
    print('The combination {} has occurred {} time(s) in history ({} extractions).\n'.format(numbers_set,num_of_occurences,total_extractions))
    one_ticket_probability(numbers)

In [109]:
check_historical_occurence([3, 41, 11, 12, 43, 14],hist['ALL NUMBERS'])

The combination {3, 41, 11, 12, 43, 14} has occurred 1 time(s) in history (3665 extractions).

Betting on these numbers you would win the big prize 1 out of 13,983,816 times!


## Multi-ticket probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [162]:
def multi_ticket_probability(successful_outcomes):
    possible_outcomes = combinations(49,6)
    probability = successful_outcomes / possible_outcomes
    print('Betting on {:,} different tickets, you would win the big prize 1 out of {:,} times (a {:.2f}% chance)!'.format(successful_outcomes,int(1/probability),probability*100))

In [163]:
multi_ticket_probability(1)
multi_ticket_probability(10)
multi_ticket_probability(100)
multi_ticket_probability(10000)
multi_ticket_probability(1000000)
multi_ticket_probability(6991908)
multi_ticket_probability(13983816)

Betting on 1 different tickets, you would win the big prize 1 out of 13,983,816 times (a 0.00% chance)!
Betting on 10 different tickets, you would win the big prize 1 out of 1,398,381 times (a 0.00% chance)!
Betting on 100 different tickets, you would win the big prize 1 out of 139,838 times (a 0.00% chance)!
Betting on 10,000 different tickets, you would win the big prize 1 out of 1,398 times (a 0.07% chance)!
Betting on 1,000,000 different tickets, you would win the big prize 1 out of 13 times (a 7.15% chance)!
Betting on 6,991,908 different tickets, you would win the big prize 1 out of 2 times (a 50.00% chance)!
Betting on 13,983,816 different tickets, you would win the big prize 1 out of 1 times (a 100.00% chance)!


## Less winning numbers

We're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

- Inside the app, the user inputs:
  - six different numbers from 1 to 49; and
  - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

As the numbers inputed by the user are irrelevant, we will only code the function with one input: the number of correct lottery numbers.

In [164]:
def probability_less_6(n):
    total_outcomes = combinations(49,6)
    successful_outcomes = combinations(6,n) * combinations(43,6-n)
    probability = successful_outcomes / total_outcomes
    print('You would get {} correct numbers 1 out of {:,} times (a {:.2f}% chance)!'.format(n,int(round(1/probability)),probability*100))

In [165]:
probability_less_6(5)
probability_less_6(4)
probability_less_6(3)
probability_less_6(2)
probability_less_6(1)

You would get 5 correct numbers 1 out of 54,201 times (a 0.00% chance)!
You would get 4 correct numbers 1 out of 1,032 times (a 0.10% chance)!
You would get 3 correct numbers 1 out of 57 times (a 1.77% chance)!
You would get 2 correct numbers 1 out of 8 times (a 13.24% chance)!
You would get 1 correct numbers 1 out of 2 times (a 41.30% chance)!
